In [1]:
import pandas as pd
import numpy as np
import requests
import bs4 as bs
import urllib.request

# 2020 from wikipedia

In [14]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2020"

In [15]:
source = urllib.request.urlopen(link).read()
soup = bs.BeautifulSoup(source,'lxml')

In [16]:
tables = soup.find_all('table',class_='wikitable sortable')

In [17]:
len(tables)

4

In [18]:
type(tables[0])

bs4.element.Tag

In [19]:
df1 = pd.read_html(str(tables[0]))[0]
df2 = pd.read_html(str(tables[1]))[0]
df3 = pd.read_html(str(tables[2]))[0]
df4 = pd.read_html(str(tables[3]).replace("'1\"\'",'"1"'))[0] 
# avoided "ValueError: invalid literal for int() with base 10: '1"'

In [20]:
df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)

In [24]:
df=df.drop(labels='Ref.', axis=1)

In [26]:
df=df.rename(columns={'.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref.':'Ref.'})

In [28]:
df

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,JANUARY,3,The Grudge,Screen Gems / Stage 6 Films / Ghost House Pict...,Nicolas Pesce (director/screenplay); Andrea Ri...,[2]
1,JANUARY,10,Underwater,20th Century Fox / TSG Entertainment / Chernin...,"William Eubank (director); Brian Duffield, Ada...",[3]
2,JANUARY,10,Like a Boss,Paramount Pictures,"Miguel Arteta (director); Sam Pitman, Adam Col...",[4]
3,JANUARY,10,Three Christs,IFC Films,Jon Avnet (director/screenplay); Eric Nazarian...,NaN
4,JANUARY,10,Inherit the Viper,Barry Films / Tycor International Film Company,Anthony Jerjen (director); Andrew Crabtree (sc...,[5]
...,...,...,...,...,...,...
270,DECEMBER,25,We Can Be Heroes,Netflix / Troublemaker Studios / Double R Prod...,Robert Rodriguez (director/screenplay); Priyan...,NaN
271,DECEMBER,25,News of the World,Universal Pictures / Perfect World Pictures,Paul Greengrass (director/screenplay); Luke Da...,NaN
272,DECEMBER,25,One Night in Miami...,Amazon Studios,Regina King (director); Kemp Powers (screenpla...,NaN
273,DECEMBER,25,Promising Young Woman,Focus Features / FilmNation Entertainment,Emerald Fennell (director/screenplay); Carey M...,NaN


In [29]:
df_2020 = df[['Title','Cast and crew']]

In [30]:
df_2020

,Title,Cast and crew
0,The Grudge,Nicolas Pesce (director/screenplay); Andrea Ri...
1,Underwater,"William Eubank (director); Brian Duffield, Ada..."
2,Like a Boss,"Miguel Arteta (director); Sam Pitman, Adam Col..."
3,Three Christs,Jon Avnet (director/screenplay); Eric Nazarian...
4,Inherit the Viper,Anthony Jerjen (director); Andrew Crabtree (sc...
...,...,...
270,We Can Be Heroes,Robert Rodriguez (director/screenplay); Priyan...
271,News of the World,Paul Greengrass (director/screenplay); Luke Da...
272,One Night in Miami...,Regina King (director); Kemp Powers (screenpla...
273,Promising Young Woman,Emerald Fennell (director/screenplay); Carey M...


In [31]:
from tmdbv3api import TMDb
import json
import requests
tmdb = TMDb()
tmdb.api_key = 'cba99650e81682e1caf42da02ec6396b'

In [32]:
from tmdbv3api import Movie
tmdb_movie = Movie() 
def get_genre(x):
    genres = []
    result = tmdb_movie.search(x)
    if not result:
      return np.NaN
    else:
      movie_id = result[0].id
      response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
      data_json = response.json()
      if data_json['genres']:
          genre_str = " " 
          for i in range(0,len(data_json['genres'])):
              genres.append(data_json['genres'][i]['name'])
          return genre_str.join(genres)
      else:
          return np.NaN

In [33]:
df_2020['genres'] = df_2020['Title'].map(lambda x: get_genre(str(x)))

/Users/shijinman/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [2]:
df_2020

NameError: name 'df_2020' is not defined

In [35]:
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]

In [36]:
df_2020['director_name'] = df_2020['Cast and crew'].map(lambda x: get_director(str(x)))

/Users/shijinman/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [37]:
def get_actor1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])

In [38]:
df_2020['actor_1_name'] = df_2020['Cast and crew'].map(lambda x: get_actor1(str(x)))

/Users/shijinman/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [39]:
def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

In [40]:
df_2020['actor_2_name'] = df_2020['Cast and crew'].map(lambda x: get_actor2(str(x)))

In [41]:
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [42]:
df_2020['actor_3_name'] = df_2020['Cast and crew'].map(lambda x: get_actor3(str(x)))


In [43]:
df_2020

,Title,Cast and crew,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,The Grudge,Nicolas Pesce (director/screenplay); Andrea Ri...,Horror Mystery Thriller,Nicolas Pesce,Andrea Riseborough,Demián Bichir,John Cho
1,Underwater,"William Eubank (director); Brian Duffield, Ada...",Action Horror Science Fiction,William Eubank,Kristen Stewart,Vincent Cassel,Jessica Henwick
2,Like a Boss,"Miguel Arteta (director); Sam Pitman, Adam Col...",Comedy,Miguel Arteta,Tiffany Haddish,Rose Byrne,Salma Hayek
3,Three Christs,Jon Avnet (director/screenplay); Eric Nazarian...,Drama,Jon Avnet,Richard Gere,Peter Dinklage,Walton Goggins
4,Inherit the Viper,Anthony Jerjen (director); Andrew Crabtree (sc...,Crime Thriller Drama,Anthony Jerjen,Josh Hartnett,Margarita Levieva,Chandler Riggs
...,...,...,...,...,...,...,...
270,We Can Be Heroes,Robert Rodriguez (director/screenplay); Priyan...,Action Fantasy Family Comedy,Robert Rodriguez,Priyanka Chopra Jonas,Pedro Pascal,YaYa Gosselin
271,News of the World,Paul Greengrass (director/screenplay); Luke Da...,Drama Western Adventure Action,Paul Greengrass,Tom Hanks,Helena Zengel,NaN
272,One Night in Miami...,Regina King (director); Kemp Powers (screenpla...,Drama,Regina King,Kingsley Ben-Adir,Eli Goree,Aldis Hodge
273,Promising Young Woman,Emerald Fennell (director/screenplay); Carey M...,Thriller Crime Drama,Emerald Fennell,Carey Mulligan,Bo Burnham,Alison Brie


In [44]:
df_2020 = df_2020.rename(columns={'Title':'movie_title'})

In [45]:
new_df20 = df_2020.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

In [46]:
new_df20.head()

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title
0,Nicolas Pesce,Andrea Riseborough,Demián Bichir,John Cho,Horror Mystery Thriller,The Grudge
1,William Eubank,Kristen Stewart,Vincent Cassel,Jessica Henwick,Action Horror Science Fiction,Underwater
2,Miguel Arteta,Tiffany Haddish,Rose Byrne,Salma Hayek,Comedy,Like a Boss
3,Jon Avnet,Richard Gere,Peter Dinklage,Walton Goggins,Drama,Three Christs
4,Anthony Jerjen,Josh Hartnett,Margarita Levieva,Chandler Riggs,Crime Thriller Drama,Inherit the Viper


In [47]:
new_df20['comb'] = new_df20['actor_1_name'] + ' ' + new_df20['actor_2_name'] + ' '+ new_df20['actor_3_name'] + ' '+ new_df20['director_name'] +' ' + new_df20['genres']

In [48]:
new_df20 = new_df20.dropna(how='any')

In [49]:
new_df20.isna().sum()

director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           0
movie_title      0
comb             0
dtype: int64

In [50]:
new_df20['movie_title'] = new_df20['movie_title'].str.lower()

In [51]:
new_df20

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Nicolas Pesce,Andrea Riseborough,Demián Bichir,John Cho,Horror Mystery Thriller,the grudge,Andrea Riseborough Demián Bichir John Cho Nico...
1,William Eubank,Kristen Stewart,Vincent Cassel,Jessica Henwick,Action Horror Science Fiction,underwater,Kristen Stewart Vincent Cassel Jessica Henwick...
2,Miguel Arteta,Tiffany Haddish,Rose Byrne,Salma Hayek,Comedy,like a boss,Tiffany Haddish Rose Byrne Salma Hayek Miguel ...
3,Jon Avnet,Richard Gere,Peter Dinklage,Walton Goggins,Drama,three christs,Richard Gere Peter Dinklage Walton Goggins Jon...
4,Anthony Jerjen,Josh Hartnett,Margarita Levieva,Chandler Riggs,Crime Thriller Drama,inherit the viper,Josh Hartnett Margarita Levieva Chandler Riggs...
...,...,...,...,...,...,...,...
269,Pete Docter,Jamie Foxx,Tina Fey,Graham Norton,Animation Family Comedy Fantasy Drama,soul,Jamie Foxx Tina Fey Graham Norton Pete Docter ...
270,Robert Rodriguez,Priyanka Chopra Jonas,Pedro Pascal,YaYa Gosselin,Action Fantasy Family Comedy,we can be heroes,Priyanka Chopra Jonas Pedro Pascal YaYa Gossel...
272,Regina King,Kingsley Ben-Adir,Eli Goree,Aldis Hodge,Drama,one night in miami...,Kingsley Ben-Adir Eli Goree Aldis Hodge Regina...
273,Emerald Fennell,Carey Mulligan,Bo Burnham,Alison Brie,Thriller Crime Drama,promising young woman,Carey Mulligan Bo Burnham Alison Brie Emerald ...


# 2021

In [54]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2021"

In [56]:
source = urllib.request.urlopen(link).read()
soup = bs.BeautifulSoup(source,'lxml')

In [57]:
tables = soup.find_all('table',class_='wikitable sortable')

In [58]:
type(tables[0])

bs4.element.Tag

In [59]:
df1 = pd.read_html(str(tables[0]))[0]
df2 = pd.read_html(str(tables[1]))[0]
df3 = pd.read_html(str(tables[2]))[0]
df4 = pd.read_html(str(tables[3]).replace("'1\"\'",'"1"'))[0] 
# avoided "ValueError: invalid literal for int() with base 10: '1"'

In [60]:
df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)

In [61]:
df

,Opening,Opening.1,Title,Production company,Cast and crew,.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref.,Ref.
0,JANUARY,1.0,Shadow in the Cloud,Vertical Entertainment,Roseanne Liang (director/screenplay); Max Land...,[2],NaN
1,JANUARY,13.0,The White Tiger,Netflix,Ramin Bahrani (director/screenplay); Adarsh Go...,NaN,NaN
2,JANUARY,14.0,Locked Down,HBO Max / Warner Bros. Pictures,Doug Liman (director); Steven Knight (screenpl...,[3],NaN
3,JANUARY,15.0,The Dig,Netflix / Clerkenwell Films,Simon Stone (director); Moira Buffini (screenp...,[4],NaN
4,JANUARY,15.0,Outside the Wire,Netflix,"Mikael Håfström (director); Rob Yescombe, Rowa...",[5],NaN
...,...,...,...,...,...,...,...
356,DECEMBER,25.0,The Tragedy of Macbeth,Apple TV+ / A24 / IAC Films,Joel Coen (director/screenplay); Denzel Washin...,NaN,[273]
357,DECEMBER,25.0,A Journal for Jordan,Columbia Pictures / Escape Artists / Bron Studios,Denzel Washington (director); Virgil Williams ...,NaN,[274]
358,DECEMBER,25.0,American Underdog,Lionsgate,"Erwin brothers (directors); Jon Erwin, David A...",NaN,[275]
359,DECEMBER,26.0,Memoria,Neon,Apichatpong Weerasethakul (director/acreenplay...,NaN,[276]


In [63]:
df=df.drop(labels='Ref.', axis=1)

In [64]:
df=df.rename(columns={'.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref.':'Ref.'})

In [65]:
df_2021 = df[['Title','Cast and crew']]

In [66]:
df_2021

,Title,Cast and crew
0,Shadow in the Cloud,Roseanne Liang (director/screenplay); Max Land...
1,The White Tiger,Ramin Bahrani (director/screenplay); Adarsh Go...
2,Locked Down,Doug Liman (director); Steven Knight (screenpl...
3,The Dig,Simon Stone (director); Moira Buffini (screenp...
4,Outside the Wire,"Mikael Håfström (director); Rob Yescombe, Rowa..."
...,...,...
356,The Tragedy of Macbeth,Joel Coen (director/screenplay); Denzel Washin...
357,A Journal for Jordan,Denzel Washington (director); Virgil Williams ...
358,American Underdog,"Erwin brothers (directors); Jon Erwin, David A..."
359,Memoria,Apichatpong Weerasethakul (director/acreenplay...


In [68]:
df_2021['genres'] = df_2021['Title'].map(lambda x: get_genre(str(x)))

/Users/shijinman/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [69]:
df_2021['director_name'] = df_2021['Cast and crew'].map(lambda x: get_director(str(x)))

/Users/shijinman/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [70]:
df_2021['actor_1_name'] = df_2021['Cast and crew'].map(lambda x: get_actor1(str(x)))

/Users/shijinman/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [71]:
df_2021['actor_2_name'] = df_2021['Cast and crew'].map(lambda x: get_actor2(str(x)))

In [72]:
df_2021['actor_3_name'] = df_2021['Cast and crew'].map(lambda x: get_actor3(str(x)))

In [73]:
df_2021

,Title,Cast and crew,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,Shadow in the Cloud,Roseanne Liang (director/screenplay); Max Land...,Horror Action War,Roseanne Liang,Chloë Grace Moretz,Taylor John Smith,Beulah Koale
1,The White Tiger,Ramin Bahrani (director/screenplay); Adarsh Go...,Drama,Ramin Bahrani,Adarsh Gourav,Rajkummar Rao,Priyanka Chopra Jonas
2,Locked Down,Doug Liman (director); Steven Knight (screenpl...,Comedy Crime Drama,Doug Liman,Anne Hathaway,Chiwetel Ejiofor,Stephen Merchant
3,The Dig,Simon Stone (director); Moira Buffini (screenp...,Drama History,Simon Stone,Carey Mulligan,Ralph Fiennes,Lily James
4,Outside the Wire,"Mikael Håfström (director); Rob Yescombe, Rowa...",Thriller Action Science Fiction,Mikael Håfström,Anthony Mackie,Damson Idris,Emily Beecham
...,...,...,...,...,...,...,...
356,The Tragedy of Macbeth,Joel Coen (director/screenplay); Denzel Washin...,Drama War,Joel Coen,Denzel Washington,Frances McDormand,Bertie Carvel
357,A Journal for Jordan,Denzel Washington (director); Virgil Williams ...,Drama Romance,Denzel Washington,Michael B. Jordan,Chanté Adams,Jalon Christian
358,American Underdog,"Erwin brothers (directors); Jon Erwin, David A...",Drama,Erwin brothers,Zachary Levi,Anna Paquin,Dennis Quaid
359,Memoria,Apichatpong Weerasethakul (director/acreenplay...,Drama Science Fiction Mystery,Apichatpong Weerasethakul (director/acreenplay...,Apichatpong Weerasethakul (director/acreenplay...,Elkin Díaz,Jeanne Balibar


In [74]:
df_2021 = df_2021.rename(columns={'Title':'movie_title'})

In [75]:
new_df21 = df_2021.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

In [76]:
new_df21

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title
0,Roseanne Liang,Chloë Grace Moretz,Taylor John Smith,Beulah Koale,Horror Action War,Shadow in the Cloud
1,Ramin Bahrani,Adarsh Gourav,Rajkummar Rao,Priyanka Chopra Jonas,Drama,The White Tiger
2,Doug Liman,Anne Hathaway,Chiwetel Ejiofor,Stephen Merchant,Comedy Crime Drama,Locked Down
3,Simon Stone,Carey Mulligan,Ralph Fiennes,Lily James,Drama History,The Dig
4,Mikael Håfström,Anthony Mackie,Damson Idris,Emily Beecham,Thriller Action Science Fiction,Outside the Wire
...,...,...,...,...,...,...
356,Joel Coen,Denzel Washington,Frances McDormand,Bertie Carvel,Drama War,The Tragedy of Macbeth
357,Denzel Washington,Michael B. Jordan,Chanté Adams,Jalon Christian,Drama Romance,A Journal for Jordan
358,Erwin brothers,Zachary Levi,Anna Paquin,Dennis Quaid,Drama,American Underdog
359,Apichatpong Weerasethakul (director/acreenplay...,Apichatpong Weerasethakul (director/acreenplay...,Elkin Díaz,Jeanne Balibar,Drama Science Fiction Mystery,Memoria


In [77]:
new_df21['comb'] = new_df21['actor_1_name'] + ' ' + new_df21['actor_2_name'] + ' '+ new_df21['actor_3_name'] + ' '+ new_df21['director_name'] +' ' + new_df21['genres']

In [78]:
new_df21.isna().sum()

director_name     0
actor_1_name      0
actor_2_name      8
actor_3_name     26
genres            0
movie_title       1
comb             26
dtype: int64

In [79]:
new_df21 = new_df21.dropna(how='any')

In [80]:
new_df21['movie_title'] = new_df21['movie_title'].str.lower()

/Users/shijinman/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [81]:
new_df21

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Roseanne Liang,Chloë Grace Moretz,Taylor John Smith,Beulah Koale,Horror Action War,shadow in the cloud,Chloë Grace Moretz Taylor John Smith Beulah Ko...
1,Ramin Bahrani,Adarsh Gourav,Rajkummar Rao,Priyanka Chopra Jonas,Drama,the white tiger,Adarsh Gourav Rajkummar Rao Priyanka Chopra Jo...
2,Doug Liman,Anne Hathaway,Chiwetel Ejiofor,Stephen Merchant,Comedy Crime Drama,locked down,Anne Hathaway Chiwetel Ejiofor Stephen Merchan...
3,Simon Stone,Carey Mulligan,Ralph Fiennes,Lily James,Drama History,the dig,Carey Mulligan Ralph Fiennes Lily James Simon ...
4,Mikael Håfström,Anthony Mackie,Damson Idris,Emily Beecham,Thriller Action Science Fiction,outside the wire,Anthony Mackie Damson Idris Emily Beecham Mika...
...,...,...,...,...,...,...,...
355,Matthew Vaughn,Ralph Fiennes,Gemma Arterton,Rhys Ifans,Action Adventure Thriller War Mystery,the king's man,Ralph Fiennes Gemma Arterton Rhys Ifans Matthe...
356,Joel Coen,Denzel Washington,Frances McDormand,Bertie Carvel,Drama War,the tragedy of macbeth,Denzel Washington Frances McDormand Bertie Car...
357,Denzel Washington,Michael B. Jordan,Chanté Adams,Jalon Christian,Drama Romance,a journal for jordan,Michael B. Jordan Chanté Adams Jalon Christian...
358,Erwin brothers,Zachary Levi,Anna Paquin,Dennis Quaid,Drama,american underdog,Zachary Levi Anna Paquin Dennis Quaid Erwin br...


In [83]:
df01 = new_df20.append(new_df21,ignore_index=True)
df01

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Nicolas Pesce,Andrea Riseborough,Demián Bichir,John Cho,Horror Mystery Thriller,the grudge,Andrea Riseborough Demián Bichir John Cho Nico...
1,William Eubank,Kristen Stewart,Vincent Cassel,Jessica Henwick,Action Horror Science Fiction,underwater,Kristen Stewart Vincent Cassel Jessica Henwick...
2,Miguel Arteta,Tiffany Haddish,Rose Byrne,Salma Hayek,Comedy,like a boss,Tiffany Haddish Rose Byrne Salma Hayek Miguel ...
3,Jon Avnet,Richard Gere,Peter Dinklage,Walton Goggins,Drama,three christs,Richard Gere Peter Dinklage Walton Goggins Jon...
4,Anthony Jerjen,Josh Hartnett,Margarita Levieva,Chandler Riggs,Crime Thriller Drama,inherit the viper,Josh Hartnett Margarita Levieva Chandler Riggs...
...,...,...,...,...,...,...,...
575,Matthew Vaughn,Ralph Fiennes,Gemma Arterton,Rhys Ifans,Action Adventure Thriller War Mystery,the king's man,Ralph Fiennes Gemma Arterton Rhys Ifans Matthe...
576,Joel Coen,Denzel Washington,Frances McDormand,Bertie Carvel,Drama War,the tragedy of macbeth,Denzel Washington Frances McDormand Bertie Car...
577,Denzel Washington,Michael B. Jordan,Chanté Adams,Jalon Christian,Drama Romance,a journal for jordan,Michael B. Jordan Chanté Adams Jalon Christian...
578,Erwin brothers,Zachary Levi,Anna Paquin,Dennis Quaid,Drama,american underdog,Zachary Levi Anna Paquin Dennis Quaid Erwin br...


# 22

In [84]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2022"

In [85]:
source = urllib.request.urlopen(link).read()
soup = bs.BeautifulSoup(source,'lxml')

In [87]:
tables = soup.find_all('table',class_='wikitable sortable')

In [88]:
df1 = pd.read_html(str(tables[0]))[0]
df2 = pd.read_html(str(tables[1]))[0]
df3 = pd.read_html(str(tables[2]))[0]
df4 = pd.read_html(str(tables[3]).replace("'1\"\'",'"1"'))[0] 
# avoided "ValueError: invalid literal for int() with base 10: '1"'

In [89]:
df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)

In [90]:
df=df.drop(labels='Ref.', axis=1)

In [91]:
df=df.rename(columns={'.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref.':'Ref.'})

In [92]:
df.head()

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,JANUARY,7.0,The 355,Universal Pictures / Freckle Films / FilmNatio...,Simon Kinberg (director/screenplay); Theresa R...,[2]
1,JANUARY,7.0,The Legend of La Llorona,Saban Films,Patricia Harris Seeley (director/screenplay); ...,[3]
2,JANUARY,7.0,The Commando,Saban Films,Asif Akbar (director); Koji Steven Sakai (scre...,[4]
3,JANUARY,7.0,American Siege,Vertical Entertainment,Edward Drake (director/screenplay); Timothy V....,[5]
4,JANUARY,14.0,Scream,Paramount Pictures / Spyglass Media Group,"Matt Bettinelli-Olpin, Tyler Gillett (director...",[6]


In [93]:
df_2022 = df[['Title','Cast and crew']]

In [94]:
df_2022

,Title,Cast and crew
0,The 355,Simon Kinberg (director/screenplay); Theresa R...
1,The Legend of La Llorona,Patricia Harris Seeley (director/screenplay); ...
2,The Commando,Asif Akbar (director); Koji Steven Sakai (scre...
3,American Siege,Edward Drake (director/screenplay); Timothy V....
4,Scream,"Matt Bettinelli-Olpin, Tyler Gillett (director..."
...,...,...
326,"Alice, Darling",Mary Nighy (director); Alanna Francis (screenp...
327,NaN,NaN
328,NaN,NaN
329,NaN,NaN


In [95]:
df_2022= df_2022.dropna(how='any')

In [96]:
df_2022['genres'] = df_2022['Title'].map(lambda x: get_genre(str(x)))

/Users/shijinman/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [97]:
df_2022['director_name'] = df_2022['Cast and crew'].map(lambda x: get_director(str(x)))

/Users/shijinman/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [98]:
df_2022['actor_1_name'] = df_2022['Cast and crew'].map(lambda x: get_actor1(str(x)))

/Users/shijinman/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [99]:
df_2022['actor_2_name'] = df_2022['Cast and crew'].map(lambda x: get_actor2(str(x)))

/Users/shijinman/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [100]:
df_2022['actor_3_name'] = df_2022['Cast and crew'].map(lambda x: get_actor3(str(x)))

/Users/shijinman/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [101]:
df_2022 = df_2022.rename(columns={'Title':'movie_title'})

In [102]:
new_df22 = df_2022.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]


In [103]:
new_df22['comb'] = new_df22['actor_1_name'] + ' ' + new_df22['actor_2_name'] + ' '+ new_df22['actor_3_name'] + ' '+ new_df22['director_name'] +' ' + new_df22['genres']


In [104]:
new_df22 = new_df22.dropna(how='any')

In [105]:
new_df22.isna().sum()

director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           0
movie_title      0
comb             0
dtype: int64

In [106]:
new_df22['movie_title'] = new_df22['movie_title'].str.lower()

In [107]:
new_df22

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Simon Kinberg,Jessica Chastain,Lupita Nyong'o,Penélope Cruz,Action Thriller,the 355,Jessica Chastain Lupita Nyong'o Penélope Cruz ...
1,Patricia Harris Seeley,Autumn Reeser,Antonio Cupo,Danny Trejo,Family Animation Fantasy,the legend of la llorona,Autumn Reeser Antonio Cupo Danny Trejo Patrici...
4,"Matt Bettinelli-Olpin, Tyler Gillett",Melissa Barrera,Mason Gooding,Jenna Ortega,Horror Mystery Thriller,scream,Melissa Barrera Mason Gooding Jenna Ortega Mat...
5,"Jennifer Kluska, Derek Drymon",Andy Samberg,Selena Gomez,Kathryn Hahn,Animation Comedy Family Fantasy Adventure,hotel transylvania: transformania,Andy Samberg Selena Gomez Kathryn Hahn Jennife...
6,Luis Prieto,Cameron Monaghan,Frank Grillo,Lilly Krug,Thriller,shattered,Cameron Monaghan Frank Grillo Lilly Krug Luis ...
...,...,...,...,...,...,...,...
322,Sarah Polley,Rooney Mara,Claire Foy,Jessie Buckley,Drama,women talking,Rooney Mara Claire Foy Jessie Buckley Sarah Po...
323,Scott Cooper,Christian Bale,Harry Melling,Gillian Anderson,Crime Thriller Mystery Horror,the pale blue eye,Christian Bale Harry Melling Gillian Anderson ...
324,Marc Forster,Tom Hanks,Mariana Treviño,Rachel Keller,Drama Comedy,a man called otto,Tom Hanks Mariana Treviño Rachel Keller Marc F...
325,Juel Taylor,John Boyega,Teyonah Parris,Kiefer Sutherland,Science Fiction Comedy Mystery,they cloned tyrone,John Boyega Teyonah Parris Kiefer Sutherland J...


In [108]:
df01 = new_df20.append(new_df21,ignore_index=True)

In [109]:
final_df = df01.append(new_df22,ignore_index=True)

In [110]:
final_df.to_csv('main_data.csv',index=False)

In [111]:
final_df 

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Nicolas Pesce,Andrea Riseborough,Demián Bichir,John Cho,Horror Mystery Thriller,the grudge,Andrea Riseborough Demián Bichir John Cho Nico...
1,William Eubank,Kristen Stewart,Vincent Cassel,Jessica Henwick,Action Horror Science Fiction,underwater,Kristen Stewart Vincent Cassel Jessica Henwick...
2,Miguel Arteta,Tiffany Haddish,Rose Byrne,Salma Hayek,Comedy,like a boss,Tiffany Haddish Rose Byrne Salma Hayek Miguel ...
3,Jon Avnet,Richard Gere,Peter Dinklage,Walton Goggins,Drama,three christs,Richard Gere Peter Dinklage Walton Goggins Jon...
4,Anthony Jerjen,Josh Hartnett,Margarita Levieva,Chandler Riggs,Crime Thriller Drama,inherit the viper,Josh Hartnett Margarita Levieva Chandler Riggs...
...,...,...,...,...,...,...,...
868,Sarah Polley,Rooney Mara,Claire Foy,Jessie Buckley,Drama,women talking,Rooney Mara Claire Foy Jessie Buckley Sarah Po...
869,Scott Cooper,Christian Bale,Harry Melling,Gillian Anderson,Crime Thriller Mystery Horror,the pale blue eye,Christian Bale Harry Melling Gillian Anderson ...
870,Marc Forster,Tom Hanks,Mariana Treviño,Rachel Keller,Drama Comedy,a man called otto,Tom Hanks Mariana Treviño Rachel Keller Marc F...
871,Juel Taylor,John Boyega,Teyonah Parris,Kiefer Sutherland,Science Fiction Comedy Mystery,they cloned tyrone,John Boyega Teyonah Parris Kiefer Sutherland J...
